In [48]:
from polymarket_analysis.data.binance import Binance
import datetime

# btc_df = Binance.load_bitcon_1h(
#     from_date=datetime.datetime(2024, 7, 1),
#     to_date=datetime.datetime(2025, 7, 23),
# )[['timestamp', 'price']]

# btc_df.to_json('/Users/kate/projects/polymarket/data/btc_hourly_data.json', 
#                orient='records', 
#                date_format='iso',
#                lines=True)

import pandas as pd
btc_df_loaded = pd.read_json('/Users/kate/projects/polymarket/data/btc_hourly_data.json', 
                            lines=True)

# Convert timestamp back to datetime if needed
btc_df_loaded['timestamp'] = pd.to_datetime(btc_df_loaded['timestamp'])

In [4]:
from polymarket_analysis.data.binance import Binance
import datetime

# btc_df = Binance.load_bitcon_5min(
#     from_date=datetime.datetime(2024, 7, 1),
#     to_date=datetime.datetime(2025, 7, 23),
# )[['timestamp', 'price']]

# print(f"Downloaded {len(btc_df)} rows")

# btc_df.to_json('/Users/kate/projects/polymarket/data/btc_5min_data.json', 
#                orient='records', 
#                date_format='iso',
#                lines=True)

import pandas as pd
btc_df_loaded = pd.read_json('/Users/kate/projects/polymarket/data/btc_5min_data.json', 
                            lines=True)

# Convert timestamp back to datetime if needed
btc_df_loaded['timestamp'] = pd.to_datetime(btc_df_loaded['timestamp'])

In [49]:
print(f"Loaded {len(btc_df_loaded)} rows")
btc_df_loaded.tail()

Loaded 9388 rows


,timestamp,price
9383,2025-07-21 13:00:00+00:00,118834.00
9384,2025-07-21 14:00:00+00:00,118828.99
9385,2025-07-21 15:00:00+00:00,118313.16
9386,2025-07-21 16:00:00+00:00,117738.44
9387,2025-07-21 17:00:00+00:00,117850.74


In [50]:
import numpy as np

btc_df_loaded['prev_timestamp'] = btc_df_loaded['timestamp'].shift(1)
btc_df_loaded['prev_price'] = btc_df_loaded['price'].shift(1)

btc_df_loaded['return'] = btc_df_loaded['price'] / btc_df_loaded['prev_price']
btc_df_loaded['log_return'] = np.log(btc_df_loaded['return'])

In [7]:
result = []

for hour in range(24):
    for interval in range(12):
        mask = (btc_df_loaded['timestamp'].dt.hour == hour) & (btc_df_loaded['timestamp'].dt.minute == interval * 5)
        if mask.any():
            # Get the data for this hour/interval
            bucket_data = btc_df_loaded.loc[mask].copy()
            
            # Drop extreme 1% values (0.5% from each tail)
            q_low = bucket_data['log_return'].quantile(0.000)
            q_high = bucket_data['log_return'].quantile(1)
            
            # Filter out extreme values
            filtered_data = bucket_data[
                (bucket_data['log_return'] >= q_low) & 
                (bucket_data['log_return'] <= q_high)
            ]
            
            # Calculate statistics on filtered data
            if len(filtered_data) > 0:
                mean_return = filtered_data['return'].mean()
                mean_log_return = filtered_data['log_return'].mean()
                median_log_return = filtered_data['log_return'].median()
                log_return_std = filtered_data['log_return'].std()
                
                result.append({
                    'hour': hour,
                    'interval': interval,
                    'mean_return': mean_return,
                    'mean_log_return': mean_log_return,
                    'median_log_return': median_log_return,
                    'log_return_std': log_return_std,
                    'n_observations': len(filtered_data),
                    'n_dropped': len(bucket_data) - len(filtered_data)
                })

result = pd.DataFrame(result)

In [ ]:
result_weekly = []

for day_of_week in range(7):  # 0=Monday, 6=Sunday
    for hour in range(24):
        for interval in range(12):
            mask = (
                (btc_df_loaded['timestamp'].dt.dayofweek == day_of_week) &
                (btc_df_loaded['timestamp'].dt.hour == hour) & 
                (btc_df_loaded['timestamp'].dt.minute == interval * 5)
            )
            if mask.any():
                # Get the data for this day_of_week/hour/interval
                bucket_data = btc_df_loaded.loc[mask].copy()
                
                # Drop extreme 1% values (0.5% from each tail)
                q_low = bucket_data['log_return'].quantile(0.005)
                q_high = bucket_data['log_return'].quantile(0.995)
                
                # Filter out extreme values
                filtered_data = bucket_data[
                    (bucket_data['log_return'] >= q_low) & 
                    (bucket_data['log_return'] <= q_high)
                ]
                
                # Calculate statistics on filtered data
                if len(filtered_data) > 0:
                    mean_return = filtered_data['return'].mean()
                    mean_log_return = filtered_data['log_return'].mean()
                    median_log_return = filtered_data['log_return'].median()
                    log_return_std = filtered_data['log_return'].std()
                    log_return_var = filtered_data['log_return'].var()
                    
                    result_weekly.append({
                        'day_of_week': day_of_week,
                        'hour': hour,
                        'interval': interval,
                        'mean_return': mean_return,
                        'mean_log_return': mean_log_return,
                        'median_log_return': median_log_return,
                        'log_return_std': log_return_std,
                        'n_observations': len(filtered_data),
                        'n_dropped': len(bucket_data) - len(filtered_data)
                    })

result_weekly = pd.DataFrame(result_weekly)

In [ ]:
window_size = 12  # 1 hour window (12 * 5min intervals)
def circular_rolling(series, window):
    """Apply rolling mean with circular boundary conditions"""
    half_window = window // 2
    
    # Create circular extension by wrapping end values to beginning
    extended = pd.concat([
        series.iloc[-half_window:],  # Last half_window values at start
        series,
        series.iloc[:half_window]    # First half_window values at end
    ], ignore_index=True)
    
    # Apply rolling and extract middle portion
    rolled = extended.rolling(window=window, center=True).mean()
    return rolled.iloc[half_window:-half_window].reset_index(drop=True)

result['mean_log_return_rolling'] = circular_rolling(result['mean_log_return'], window_size)
result['median_log_return_rolling'] = circular_rolling(result['median_log_return'], window_size)
result['log_return_std_rolling'] = circular_rolling(result['log_return_std'], window_size)

In [28]:
result_weekly['mean_log_return_rolling'] = circular_rolling(result_weekly['mean_log_return'], window_size)
result_weekly['median_log_return_rolling'] = circular_rolling(result_weekly['median_log_return'], window_size)
result_weekly['log_return_std_rolling'] = circular_rolling(result_weekly['log_return_std'], window_size)
result_weekly['log_return_std_rolling_daily'] = circular_rolling(result_weekly['log_return_std'], window_size*24)

In [33]:
periodic_data = []
periodic_weekly_data = []
date_range = pd.date_range(start=start_date.date(), end=end_date.date(), freq='D')

for date in date_range:
    day_of_week = date.dayofweek
    
    # Daily periodic data
    for _, row in result.iterrows():
        periodic_timestamp = pd.Timestamp.combine(
            date, 
            time(hour=int(row['hour']), minute=int(row['interval'] * 5))
        )
        periodic_data.append({
            'timestamp': periodic_timestamp,
            'periodic_median_log_return': row['median_log_return'],
            'periodic_log_return_std_rolling': row['log_return_std_rolling']
        })
    
    # Weekly periodic data
    weekly_rows = result_weekly[result_weekly['day_of_week'] == day_of_week]
    for _, row in weekly_rows.iterrows():
        periodic_timestamp = pd.Timestamp.combine(
            date, 
            time(hour=int(row['hour']), minute=int(row['interval'] * 5))
        )
        periodic_weekly_data.append({
            'timestamp': periodic_timestamp,
            'periodic_weekly_median_log_return': row['median_log_return'],
            'periodic_weekly_log_return_std_rolling': row['log_return_std_rolling'],
            'periodic_weekly_log_return_std_rolling_daily': row['log_return_std_rolling_daily']
        })

periodic_df = pd.DataFrame(periodic_data)
periodic_weekly_df = pd.DataFrame(periodic_weekly_data)

In [ ]:
import numpy as np

# Define lookback window (1 week = 7*24*12 = 2016 intervals)
lookback_window = 1 * 24 * 12  # 1 day of 5-min intervals

# Calculate rolling standard deviation of actual data (equivalent to last_week_log_return_std for each timestamp)
btc_recent['rolling_actual_std'] = btc_recent['log_return'].rolling(
    window=lookback_window, 
    min_periods=lookback_window//2
).std()

# Add template std to btc_recent first
periodic_lookup = {}
for _, row in periodic_weekly_df.iterrows():
    key = (row['timestamp'].dayofweek, row['timestamp'].hour, row['timestamp'].minute)
    periodic_lookup[key] = row['periodic_weekly_log_return_std_rolling']

btc_recent['template_std'] = btc_recent['timestamp'].apply(
    lambda ts: periodic_lookup.get((ts.dayofweek, ts.hour, ts.minute), np.nan)
)

# Calculate rolling RMS of template std for each timestamp (same window as actual data)
btc_recent['template_rms_std_rolling'] = btc_recent['template_std'].rolling(
    window=lookback_window,
    min_periods=lookback_window//2
).apply(lambda x: np.sqrt(np.mean(x**2)), raw=True)


# Calculate time-varying scaling factor using rolling template RMS
btc_recent['scaling_factor'] = btc_recent['rolling_actual_std'] / btc_recent['template_rms_std_rolling']

# Apply scaling to get time-varying scaled template std
btc_recent['scaled_periodic_std'] = btc_recent['template_std'] * btc_recent['scaling_factor']

# Fill any NaN values with the static scaling factor from cell 36
btc_recent['scaling_factor'] = btc_recent['scaling_factor']
btc_recent['scaled_periodic_std'] = btc_recent['scaled_periodic_std']

print(f"Time-varying scaling factor range: {btc_recent['scaling_factor'].min():.4f} to {btc_recent['scaling_factor'].max():.4f}")
print(f"Time-varying scaling factor mean: {btc_recent['scaling_factor'].mean():.4f}")

Time-varying scaling factor range: 0.3863 to 2.3465
Time-varying scaling factor mean: 0.8256


,timestamp,price,prev_timestamp,prev_price,return,log_return,log_return_rolling_std_1h,log_return_rolling_std_4h,log_return_rolling_std_day,log_return_rolling_median,rolling_actual_std,template_std,scaling_factor,scaled_periodic_std,template_rms_std_rolling
94284,2025-05-23 22:00:00+00:00,108342.74,2025-05-23 21:55:00+00:00,108282.89,1.000553,0.000553,NaN,NaN,NaN,NaN,NaN,0.000964,NaN,NaN,NaN
94285,2025-05-23 22:00:00+00:00,108342.74,2025-05-23 22:00:00+00:00,108342.74,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.000964,NaN,NaN,NaN
94286,2025-05-23 22:05:00+00:00,108348.42,2025-05-23 22:00:00+00:00,108342.74,1.000052,0.000052,NaN,NaN,NaN,NaN,NaN,0.000960,NaN,NaN,NaN
94287,2025-05-23 22:10:00+00:00,108492.30,2025-05-23 22:05:00+00:00,108348.42,1.001328,0.001327,NaN,NaN,NaN,NaN,NaN,0.000954,NaN,NaN,NaN
94288,2025-05-23 22:15:00+00:00,108424.00,2025-05-23 22:10:00+00:00,108492.30,0.999370,-0.000630,NaN,NaN,NaN,NaN,NaN,0.000935,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111580,2025-07-22 21:40:00+00:00,119479.89,2025-07-22 21:35:00+00:00,119501.33,0.999821,-0.000179,NaN,NaN,NaN,NaN,0.001223,0.001318,0.905531,0.001194,0.001351
111581,2025-07-22 21:45:00+00:00,119480.32,2025-07-22 21:40:00+00:00,119479.89,1.000004,0.000004,NaN,NaN,NaN,NaN,0.001222,0.001305,0.905231,0.001181,0.001350
111582,2025-07-22 21:50:00+00:00,119546.56,2025-07-22 21:45:00+00:00,119480.32,1.000554,0.000554,NaN,NaN,NaN,NaN,0.001222,0.001314,0.905687,0.001190,0.001349
111583,2025-07-22 21:55:00+00:00,119400.00,2025-07-22 21:50:00+00:00,119546.56,0.998774,-0.001227,NaN,NaN,NaN,NaN,0.001224,0.001299,0.907642,0.001179,0.001349


In [42]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Plot scaling factor over time
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add scaling factor
fig.add_trace(go.Scatter(
    x=btc_recent['timestamp'],
    y=btc_recent['scaling_factor'],
    mode='lines',
    name='Time-varying Scaling Factor',
    line=dict(color='blue', width=2)
), secondary_y=False)

# Add rolling actual std for context
fig.add_trace(go.Scatter(
    x=btc_recent['timestamp'],
    y=btc_recent['rolling_actual_std'],
    mode='lines',
    name='Rolling Actual Std',
    line=dict(color='red', width=1),
    opacity=0.7
), secondary_y=True)

# Add template std for context
fig.add_trace(go.Scatter(
    x=btc_recent['timestamp'],
    y=btc_recent['template_std'],
    mode='lines',
    name='Template Std',
    line=dict(color='green', width=1),
    opacity=0.7
), secondary_y=True)

# Add scaled periodic std
fig.add_trace(go.Scatter(
    x=btc_recent['timestamp'],
    y=btc_recent['scaled_periodic_std'],
    mode='lines',
    name='Scaled Periodic Std',
    line=dict(color='purple', width=2)
), secondary_y=True)

# Update axes
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Scaling Factor", secondary_y=False)
fig.update_yaxes(title_text="Standard Deviation", secondary_y=True)

# Update layout
fig.update_layout(
    title='Time-varying Scaling Factor and Standard Deviations',
    showlegend=True,
    hovermode='x unified',
    height=600
)

fig.show(renderer="browser")

In [52]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Calculate rolling mean log returns for different windows
# 1 day = 1*24*12 = 288 intervals (5-min intervals)
# 7 days = 7*24*12 = 2016 intervals  
# 1 month = 30*24*12 = 8640 intervals

mean_df = btc_df_loaded.copy()

mean_df['mean_log_return_1d'] = mean_df['log_return'].rolling(
    window=24, 
    min_periods=12
).mean()

mean_df['mean_log_return_7d'] = mean_df['log_return'].rolling(
    window=7*24, 
    min_periods=7*24//2
).mean()

mean_df['mean_log_return_1m'] = mean_df['log_return'].rolling(
    window=30*24, 
    min_periods=30*24//2
).mean()

# Create plot
fig = go.Figure()

# Add rolling mean log returns
fig.add_trace(go.Scatter(
    x=mean_df['timestamp'],
    y=mean_df['mean_log_return_1d'],
    mode='lines',
    name='Rolling Mean Log Return (1 day)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=mean_df['timestamp'],
    y=mean_df['mean_log_return_7d'],
    mode='lines',
    name='Rolling Mean Log Return (7 days)',
    line=dict(color='red', width=2)
))

fig.add_trace(go.Scatter(
    x=mean_df['timestamp'],
    y=mean_df['mean_log_return_1m'],
    mode='lines',
    name='Rolling Mean Log Return (1 month)',
    line=dict(color='green', width=2)
))

# Add horizontal line at zero
fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="grey")

# Add overall mean for reference
overall_mean = mean_df['log_return'].mean()
fig.add_hline(
    y=overall_mean,
    line_width=1,
    line_dash="dot",
    line_color="black",
    annotation_text=f"Overall Mean: {overall_mean:.6f}"
)

# Update layout
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Mean Log Return")

fig.update_layout(
    title='Bitcoin Rolling Mean Log Returns - Multiple Time Windows',
    showlegend=True,
    hovermode='x unified',
    height=600
)

fig.show(renderer="browser")

# Print summary statistics
print(f"Rolling mean log return statistics:")
print(f"1 day window - mean: {mean_df['mean_log_return_1d'].mean():.6f}, std: {mean_df['mean_log_return_1d'].std():.6f}")
print(f"7 day window - mean: {mean_df['mean_log_return_7d'].mean():.6f}, std: {mean_df['mean_log_return_7d'].std():.6f}")
print(f"1 month window - mean: {mean_df['mean_log_return_1m'].mean():.6f}, std: {mean_df['mean_log_return_1m'].std():.6f}")

Rolling mean log return statistics:
1 day window - mean: 0.000067, std: 0.001041
7 day window - mean: 0.000067, std: 0.000375
1 month window - mean: 0.000068, std: 0.000168


In [45]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import time

end_date = btc_df_loaded['timestamp'].max()
start_date = end_date - datetime.timedelta(days=160)
btc_recent = btc_df_loaded[btc_df_loaded['timestamp'] >= start_date].copy()

# Calculate rolling statistics
window_size_rolling = 12  # 1 hour window
btc_recent['log_return_rolling_std_1h'] = btc_recent['log_return'].rolling(window=window_size_rolling, center=True).std()
btc_recent['log_return_rolling_std_4h'] = btc_recent['log_return'].rolling(window=window_size_rolling*4, center=True).std()
btc_recent['log_return_rolling_std_day'] = btc_recent['log_return'].rolling(window=12*24, center=True).std()
btc_recent['log_return_rolling_median'] = btc_recent['log_return'].rolling(window=window_size_rolling, center=True).median()

# Create plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add rolling std from actual data
fig.add_trace(go.Scatter(
    x=btc_recent['timestamp'],
    y=btc_recent['log_return_rolling_std_1h'],
    mode='lines',
    name='Rolling Log Return Std 1h (Actual)',
    line=dict(color='blue', width=1),
    opacity=0.7
), secondary_y=True)

fig.add_trace(go.Scatter(
    x=btc_recent['timestamp'],
    y=btc_recent['log_return_rolling_std_4h'],
    mode='lines',
    name='Rolling Log Return Std 4h (Actual)',
    line=dict(color='darkblue', width=1),
    opacity=0.7
), secondary_y=True)

fig.add_trace(go.Scatter(
    x=btc_recent['timestamp'],
    y=btc_recent['log_return_rolling_std_day'],
    mode='lines',
    name='Rolling Log Return Std Daily (Actual)',
    line=dict(color='black', width=1),
    opacity=0.7
), secondary_y=True)

# Add periodic std pattern (daily)
fig.add_trace(go.Scatter(
    x=periodic_df['timestamp'],
    y=periodic_df['periodic_log_return_std_rolling'],
    mode='lines',
    name='Periodic Log Return Std Rolling (Daily Pattern)',
    line=dict(color='darkred', width=2),
    opacity=0.8
), secondary_y=True)

fig.add_trace(go.Scatter(
    x=periodic_weekly_df['timestamp'],
    y=periodic_weekly_df['periodic_weekly_log_return_std_rolling'],
    mode='lines',
    name='Periodic Log Return Std Rolling (Weekly Pattern)',
    line=dict(color='purple', width=2),
    opacity=0.8
), secondary_y=True)

fig.add_trace(go.Scatter(
    x=periodic_weekly_df['timestamp'],
    y=periodic_weekly_df['scaled_periodic_std'],
    mode='lines',
    name='Periodic Log Return Std Rolling Scaled (Weekly Pattern)',
    line=dict(color='darkgreen', width=2),
    opacity=0.8
), secondary_y=True)

fig.add_trace(go.Scatter(
    x=periodic_weekly_df['timestamp'],
    y=periodic_weekly_df['periodic_weekly_log_return_std_rolling_daily'],
    mode='lines',
    name='Periodic Log Return Std Rolling Daily (Weekly Pattern)',
    line=dict(color='cyan', width=2),
    opacity=0.8
), secondary_y=True)


# Update layout
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Median Log Return", secondary_y=False)
fig.update_yaxes(title_text="Log Return Standard Deviation", secondary_y=True)

fig.update_layout(
    title='Bitcoin Rolling vs Periodic Statistics - Last 60 Days',
    showlegend=True,
    hovermode='x unified',
    height=600
)

fig.show(renderer="browser")

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import time

# Create subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

result['time_of_day'] = result.apply(
    lambda row: time(hour=int(row['hour']), 
                     minute=int(row['interval'] * 5)), 
    axis=1
)

# Add original mean log return trace on primary y-axis
fig.add_trace(go.Scatter(
    x=result['time_of_day'],
    y=result['mean_log_return'],
    mode='lines+markers',
    name='Mean Log Return (5-min)',
    line=dict(color='lightblue', width=1),
    marker=dict(size=2),
    opacity=0.6
), secondary_y=False)

# Add rolling average of mean log return
fig.add_trace(go.Scatter(
    x=result['time_of_day'],
    y=result['mean_log_return_rolling'],
    mode='lines',
    name=f'Mean Log Return ({window_size*5}min Rolling)',
    line=dict(color='blue', width=3)
), secondary_y=False)

# Add original mean log return trace on primary y-axis
fig.add_trace(go.Scatter(
    x=result['time_of_day'],
    y=result['median_log_return'],
    mode='lines+markers',
    name='Median Log Return (5-min)',
    line=dict(color='lightgreen', width=1),
    marker=dict(size=2),
    opacity=0.6
), secondary_y=False)

# Add rolling average of mean log return
fig.add_trace(go.Scatter(
    x=result['time_of_day'],
    y=result['median_log_return_rolling'],
    mode='lines',
    name=f'Median Log Return ({window_size*5}min Rolling)',
    line=dict(color='green', width=3)
), secondary_y=False)

# Add original log return std trace on secondary y-axis
fig.add_trace(go.Scatter(
    x=result['time_of_day'],
    y=result['log_return_std'],
    mode='lines+markers',
    name='Log Return Std (5-min)',
    line=dict(color='lightcoral', width=1),
    marker=dict(size=2),
    opacity=0.6
), secondary_y=True)

# Add rolling average of log return std
fig.add_trace(go.Scatter(
    x=result['time_of_day'],
    y=result['log_return_std_rolling'],
    mode='lines',
    name=f'Log Return Std ({window_size*5}min Rolling)',
    line=dict(color='red', width=3)
), secondary_y=True)

# Update axes labels
fig.update_xaxes(title_text="Time of Day")
fig.update_yaxes(title_text="Mean Log Return", secondary_y=False)
fig.update_yaxes(title_text="Log Return Standard Deviation", secondary_y=True)

# Update layout
fig.update_layout(
    title='Bitcoin 5-Minute Interval Log Return Statistics Throughout the Day',
    showlegend=True,
    hovermode='x unified',
    height=600
)

fig.show(renderer="browser")

In [26]:
hours = []
for h in range(24):
    h_df = btc_df_loaded[btc_df_loaded['timestamp'].dt.hour == h]
    mu = h_df['log_return'].mean()
    sigma = h_df['log_return'].std()
    avg_return = h_df['return'].mean()
    print(f"Hour {h}: mu={mu*1000000}, sigma={sigma*1000}, avg_return={avg_return}")
    hours.append({
        'hour': h,
        'mu': mu,
        'sigma': sigma})

Hour 0: mu=-131.28775614784396, sigma=5.811852673953044, avg_return=0.9998855196700208
Hour 1: mu=43.09769120372629, sigma=6.297095804622536, avg_return=1.0000628383601868
Hour 2: mu=339.4196916474739, sigma=4.9271281908540585, avg_return=1.0003516066307003
Hour 3: mu=66.44708621770543, sigma=4.210721423420913, avg_return=1.0000753026658802
Hour 4: mu=-22.392614547109194, sigma=3.9649395255185302, avg_return=0.9999854536926049
Hour 5: mu=264.08816690906264, sigma=3.51554699034802, avg_return=1.0002702883280856
Hour 6: mu=-122.36095562870598, sigma=4.692721522265116, avg_return=0.999888662147742
Hour 7: mu=-65.70587079337, sigma=4.080727694159952, avg_return=0.9999426016270319
Hour 8: mu=195.18872121342469, sigma=4.293422900220435, avg_return=1.0002044089036717
Hour 9: mu=400.73577099140147, sigma=3.8476305277275467, avg_return=1.0004082055959076
Hour 10: mu=-29.468400486926413, sigma=3.803919870778834, avg_return=0.999977736902016
Hour 11: mu=70.96120088963123, sigma=3.730175916567771,

In [27]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Extract hours, mu, and sigma from the hours list
hours_df = pd.DataFrame(hours)

# Add mu (mean) on primary y-axis
fig.add_trace(go.Scatter(
    x=hours_df['hour'],
    y=hours_df['mu'],
    mode='lines+markers',
    name='Mu (Mean Log Return)',
    line=dict(color='blue', width=2),
    marker=dict(size=6)
), secondary_y=False)

# Add sigma (std) on secondary y-axis
fig.add_trace(go.Scatter(
    x=hours_df['hour'],
    y=hours_df['sigma'],
    mode='lines+markers',
    name='Sigma (Std Log Return)',
    line=dict(color='red', width=2),
    marker=dict(size=6)
), secondary_y=True)

# Update layout
fig.update_xaxes(title_text="Hour")
fig.update_yaxes(title_text="Mu (Mean Log Return)", secondary_y=False)
fig.update_yaxes(title_text="Sigma (Std Log Return)", secondary_y=True)

fig.update_layout(
    title='Bitcoin Hourly Log Return Statistics',
    showlegend=True,
    hovermode='x unified'
)

fig.show()

In [42]:
# from polymarket_analysis.data.binance import Binance
# import datetime

# btc_df_day = Binance.load_bitcon_1d(
#     from_date=datetime.datetime(2024, 7, 1),
#     to_date=datetime.datetime(2025, 7, 23),
# )[['timestamp', 'price']]

# btc_df_loaded = btc_df_day.copy()
# btc_df_loaded['prev_timestamp'] = btc_df_loaded['timestamp'].shift(1)
# btc_df_loaded['prev_price'] = btc_df_loaded['price'].shift(1)
# btc_df_loaded['timestamp_et'] = btc_df_loaded['timestamp'].dt.tz_convert('US/Eastern')

# btc_df_loaded['return'] = btc_df_loaded['price'] / btc_df_loaded['prev_price']
# btc_df_loaded['log_return'] = np.log(btc_df_loaded['return'])

In [94]:
from polymarket_analysis.data.binance import Binance
import datetime

btc_df_day = Binance.load_bitcon_5min(
    from_date=datetime.datetime(2025, 5, 1),
    to_date=datetime.datetime(2025, 7, 23),
)[['timestamp', 'price']]

btc_df_loaded = btc_df_day.copy()
btc_df_loaded['prev_timestamp'] = btc_df_loaded['timestamp'].shift(1)
btc_df_loaded['prev_price'] = btc_df_loaded['price'].shift(1)
btc_df_loaded['timestamp_et'] = btc_df_loaded['timestamp'].dt.tz_convert('US/Eastern')

btc_df_loaded['return'] = btc_df_loaded['price'] / btc_df_loaded['prev_price']
btc_df_loaded['log_return'] = np.log(btc_df_loaded['return'])

In [95]:
btc_df_loaded['average_hour_sigma'] = btc_df_loaded['timestamp'].dt.hour.apply(lambda h: hours[h]['sigma'])

In [101]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Calculate rolling average and rolling variance of log_return
period = '5min'
window_size = 12
btc_df_loaded['log_return_rolling'] = btc_df_loaded['log_return'].rolling(window=window_size).mean()
btc_df_loaded['log_return_rolling_sigma_hourly'] = btc_df_loaded['log_return'].rolling(window=window_size).std()*np.sqrt(12)*5

# Create subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add original log returns
fig.add_trace(go.Scatter(
    x=btc_df_loaded['timestamp'],
    y=btc_df_loaded['log_return'],
    mode='lines',
    name='Log Return',
    line=dict(color='lightblue', width=1),
    opacity=0.6
), secondary_y=False)

# Add rolling average
fig.add_trace(go.Scatter(
    x=btc_df_loaded['timestamp'],
    y=btc_df_loaded['log_return_rolling'],
    mode='lines',
    name=f'{window_size}-{period} Rolling Average Log Return',
    line=dict(color='red', width=2)
), secondary_y=False)

# Add original log return average (overall mean)
overall_log_return_mean = btc_df_loaded['log_return'].mean()
fig.add_hline(
    y=overall_log_return_mean,
    line_width=2,
    line_dash="solid",
    line_color="darkred",
    opacity=0.6,
    annotation_text="Overall Log Return Mean",
    secondary_y=False
)

fig.add_trace(go.Scatter(
    x=btc_df_loaded['timestamp'],
    y=btc_df_loaded['log_return'],
    mode='lines',
    name=f'Log Return',
    opacity=0.4,
    line=dict(color='darkred', width=2)
), secondary_y=False)

# Add rolling variance on secondary y-axis
fig.add_trace(go.Scatter(
    x=btc_df_loaded['timestamp'],
    y=btc_df_loaded['log_return_rolling_sigma_hourly'],
    mode='lines',
    name=f'{window_size}-{period} Rolling Sigma',
    line=dict(color='green', width=2)
), secondary_y=True)

# Add rolling variance on secondary y-axis
fig.add_trace(go.Scatter(
    x=btc_df_loaded['timestamp'],
    y=btc_df_loaded['average_hour_sigma'],
    mode='lines',
    name=f'Average Hour Sigma',
    opacity=0.8,
    line=dict(color='grey', width=2)
), secondary_y=True)

# Add original log return variance (overall variance)
overall_log_return_var = btc_df_loaded['log_return'].std()
fig.add_hline(
    y=overall_log_return_var,
    line_width=1,
    line_dash="solid", 
    line_color="darkgreen",
    opacity=0.4,
    annotation_text="Overall Log Return Sigma",
    secondary_y=True
)


# Add horizontal line at zero
fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="grey", secondary_y=False)

# Add vertical lines for start of week (UTC midnight Sunday-Monday)
# Convert back to UTC for week calculation
btc_df_loaded['timestamp_utc'] = btc_df_loaded['timestamp']
btc_df_loaded['hour_utc'] = btc_df_loaded['timestamp_utc'].dt.hour
btc_df_loaded['minute_utc'] = btc_df_loaded['timestamp_utc'].dt.minute

midnight_starts = btc_df_loaded[
    (btc_df_loaded['hour_utc'] == 0) & 
    (btc_df_loaded['minute_utc'] == 0)
]

for _, row in midnight_starts.iterrows():
    # is_monday = row['weekday'] == 0
    
    fig.add_vline(
        x=row['timestamp_et'], 
        line_width=2 if is_monday else 1, 
        line_dash="solid" if is_monday else "dot", 
        line_color="red" if is_monday else "blue",
        opacity=0.8 if is_monday else 0.5
    )

fig.show(renderer="browser")

In [48]:
btc_df_loaded.head()

,timestamp,price,prev_timestamp,prev_price,timestamp_et,return,log_return,log_return_rolling,log_return_rolling_var,timestamp_utc,weekday,hour_utc
0,2025-04-30 22:00:00+00:00,94613.99,NaT,NaN,2025-04-30 18:00:00-04:00,NaN,NaN,NaN,NaN,2025-04-30 22:00:00+00:00,2,22
1,2025-04-30 22:05:00+00:00,94687.46,2025-04-30 22:00:00+00:00,94613.99,2025-04-30 18:05:00-04:00,1.000777,0.000776,NaN,NaN,2025-04-30 22:05:00+00:00,2,22
2,2025-04-30 22:10:00+00:00,94584.66,2025-04-30 22:05:00+00:00,94687.46,2025-04-30 18:10:00-04:00,0.998914,-0.001086,NaN,NaN,2025-04-30 22:10:00+00:00,2,22
3,2025-04-30 22:15:00+00:00,94520.20,2025-04-30 22:10:00+00:00,94584.66,2025-04-30 18:15:00-04:00,0.999318,-0.000682,NaN,NaN,2025-04-30 22:15:00+00:00,2,22
4,2025-04-30 22:20:00+00:00,94489.34,2025-04-30 22:15:00+00:00,94520.20,2025-04-30 18:20:00-04:00,0.999674,-0.000327,NaN,NaN,2025-04-30 22:20:00+00:00,2,22


In [49]:
import numpy as np

def p_larger(hours_ahead: float, current_price: float, target_price: float) -> float:

    # Calculate drift and volatility
    log_drift = (self.mu - 0.5 * self.sigma**2) * hours_ahead
    log_volatility = self.sigma * np.sqrt(hours_ahead)
            
    # Calculate log ratio: ln(target_price / current_price)
    log_target_ratio = np.log(target_price / current_price)